<a href="https://colab.research.google.com/github/Prasad020202/Speech_emotion_recognition/blob/master/colab_notebook_recognition0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}



In [ ]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result


In [ ]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/content/drive/MyDrive/ColabNotebooks/archive/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          basename_parts = basename.split("-")
          if len(basename_parts) < 3:
              continue
          emotion = int2emotion[basename_parts[2]]

          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
          y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)


In [ ]:
X_train, X_test, y_train, y_test = load_data(test_size=0.2)

print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])

03-01-01-01-02-02-18.wav
03-01-03-02-02-02-18.wav
03-01-02-01-01-01-18.wav
03-01-04-01-02-02-18.wav
03-01-04-01-02-01-18.wav
03-01-04-01-01-02-18.wav
03-01-03-02-01-02-18.wav
03-01-03-02-01-01-18.wav
03-01-02-02-02-02-18.wav
03-01-01-01-01-01-18.wav
03-01-02-01-01-02-18.wav
03-01-03-01-02-02-18.wav
03-01-02-02-02-01-18.wav
03-01-04-02-01-01-18.wav
03-01-04-01-01-01-18.wav
03-01-02-02-01-02-18.wav
03-01-04-02-01-02-18.wav
03-01-01-01-01-02-18.wav
03-01-03-01-01-01-18.wav
03-01-03-01-01-02-18.wav
03-01-01-01-02-01-18.wav
03-01-03-01-02-01-18.wav
03-01-03-02-02-01-18.wav
03-01-02-02-01-01-18.wav
03-01-02-01-02-02-18.wav
03-01-02-01-02-01-18.wav
03-01-07-02-01-02-18.wav
03-01-05-01-01-01-18.wav
03-01-08-02-02-02-18.wav
03-01-07-01-02-02-18.wav
03-01-05-02-02-01-18.wav
03-01-07-02-02-01-18.wav
03-01-07-02-02-02-18.wav
03-01-05-01-02-02-18.wav
03-01-08-01-02-01-18.wav
03-01-08-01-01-02-18.wav
03-01-08-01-02-02-18.wav
03-01-08-02-01-02-18.wav
03-01-07-02-01-01-18.wav
03-01-07-01-02-01-18.wav


In [ ]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((56, 180), (56,), (15, 180), (15,))

In [ ]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [ ]:
x_traincnn.shape,x_testcnn.shape

((56, 180, 1), (15, 180, 1))

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=None, decay=0.0)



In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 180, 128)          768       
                                                                 
 activation (Activation)     (None, 180, 128)          0         
                                                                 
 dropout (Dropout)           (None, 180, 128)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 22, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_1 (Activation)   (None, 22, 128)           0         
                                                        

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Epoch 1/500
3/3 [==============================] - 2s 155ms/step - loss: 8.2259 - accuracy: 0.0893 - val_loss: 2.9841 - val_accuracy: 0.2000
Epoch 2/500
3/3 [==============================] - 0s 82ms/step - loss: 4.9069 - accuracy: 0.1429 - val_loss: 2.0915 - val_accuracy: 0.3333
Epoch 3/500
3/3 [==============================] - 0s 83ms/step - loss: 3.1907 - accuracy: 0.2321 - val_loss: 1.8737 - val_accuracy: 0.2000
Epoch 4/500
3/3 [==============================] - 0s 84ms/step - loss: 3.3390 - accuracy: 0.2500 - val_loss: 1.9626 - val_accuracy: 0.4000
Epoch 5/500
3/3 [==============================] - 0s 106ms/step - loss: 3.6875 - accuracy: 0.2143 - val_loss: 2.2528 - val_accuracy: 0.2667
Epoch 6/500
3/3 [==============================] - 0s 94ms/step - loss: 3.7421 - accuracy: 0.2321 - val_loss: 2.2108 - val_accuracy: 0.2000
Epoch 7/500
3/3 [==============================] - 0s 93ms/step - loss: 3.4776 - accuracy: 0.2679 - val_loss: 1.7137 - val_accuracy: 0.2667
Epoch 8/500
3/3 [=

In [ ]:
# Specify the path to save your model
model_path = '/content/drive/MyDrive/ColabNotebooks/my_modeltbnc.h5'

# Save your model to the specified path
model.save(model_path)
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/my_modeltbnc.h5')
loaded_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
em=['happy','sad','neutral','angry']

In [ ]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1/1 [==============================] - 0s 30ms/step - loss: 0.6552 - accuracy: 0.6667
Restored model, accuracy: 66.67%


In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#defined your model as `model`

# get class probabilities for test data
probabilities = model.predict(x_testcnn)

# get predicted classes by taking class with highest probability
predicted_classes = np.argmax(probabilities, axis=1)

# print classification report and confusion matrix
print(classification_report(y_test, predicted_classes))
print(confusion_matrix(y_test, predicted_classes))


1/1 [==============================] - 0s 123ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75         5
         1.0       0.33      0.50      0.40         2
         3.0       1.00      0.50      0.67         4
         4.0       0.57      1.00      0.73         4

    accuracy                           0.67        15
   macro avg       0.73      0.65      0.64        15
weighted avg       0.80      0.67      0.68        15

[[3 0 0 2]
 [0 1 0 1]
 [0 2 2 0]
 [0 0 0 4]]


In [ ]:
import tensorflow as tf
import numpy as np
import librosa

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/my_modeltbnc.h5')

# Define the emotions
emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

# Define a function to extract features from an audio file
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    audio_file, sample_rate = librosa.load(file_path, sr=22050)
    if chroma:
        chromagram = librosa.feature.chroma_stft(y=audio_file, sr=sample_rate)
    if mfcc:
        mfccs = librosa.feature.mfcc(y=audio_file, sr=sample_rate, n_mfcc=40)
    if mel:
        mel_spectrogram = librosa.feature.melspectrogram(y=audio_file, sr=sample_rate)
    result = np.array([])
    if mfcc:
        result = np.hstack((result, np.mean(mfccs, axis=1)))
    if chroma:
        result = np.hstack((result, np.mean(chromagram, axis=1)))
    if mel:
        result = np.hstack((result, np.mean(mel_spectrogram, axis=1)))
    return result


# Load the audio file and extract features
filename = "/content/drive/MyDrive/ColabNotebooks/archive/Actor_02/03-01-01-01-01-01-02.wav"
features = extract_features(filename)

# Reshape the features for use in the model
features = features.reshape(1, -1, 1)

# Predict the emotion
probabilities = model.predict(features)[0]
predicted_emotion = emotions[np.argmax(probabilities)]

# Print the predicted emotion
print("Predicted emotion:", predicted_emotion)


1/1 [==============================] - 0s 97ms/step
Predicted emotion: calm


In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

tm = Sequential()

tm.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
tm.add(Activation('relu'))
tm.add(Dropout(0.1))
tm.add(MaxPooling1D(pool_size=(8)))

tm.add(Conv1D(128, 5,padding='same',))                  #2
tm.add(Activation('relu'))
tm.add(MaxPooling1D(pool_size=(8)))
tm.add(Dropout(0.1))

tm.add(Conv1D(128, 5,padding='same',))                  #3
tm.add(Activation('relu'))
tm.add(Dropout(0.1))

tm.add(Flatten())
tm.add(Dense(8))                                        #4
tm.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(learning_rate=0.00005,epsilon=None,rho=0.9,decay=0.0)

In [ ]:
tm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_3 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_2 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_4 (Activation)   (None, 22, 128)           0         
                                                      

In [ ]:
tm.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [ ]:
tmhistory=tm.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Epoch 1/500
3/3 [==============================] - 1s 144ms/step - loss: 12.1782 - accuracy: 0.0893 - val_loss: 3.1268 - val_accuracy: 0.3333
Epoch 2/500
3/3 [==============================] - 0s 37ms/step - loss: 8.0972 - accuracy: 0.1071 - val_loss: 2.0927 - val_accuracy: 0.4667
Epoch 3/500
3/3 [==============================] - 0s 38ms/step - loss: 5.1361 - accuracy: 0.2857 - val_loss: 1.8539 - val_accuracy: 0.2667
Epoch 4/500
3/3 [==============================] - 0s 40ms/step - loss: 5.3453 - accuracy: 0.2679 - val_loss: 1.3715 - val_accuracy: 0.5333
Epoch 5/500
3/3 [==============================] - 0s 48ms/step - loss: 5.5377 - accuracy: 0.1607 - val_loss: 1.2758 - val_accuracy: 0.2667
Epoch 6/500
3/3 [==============================] - 0s 47ms/step - loss: 4.6097 - accuracy: 0.1964 - val_loss: 1.4851 - val_accuracy: 0.1333
Epoch 7/500
3/3 [==============================] - 0s 36ms/step - loss: 4.1625 - accuracy: 0.3393 - val_loss: 1.5653 - val_accuracy: 0.1333
Epoch 8/500
3/3 [=

In [ ]:
loss, acc = tm.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1/1 [==============================] - 0s 30ms/step - loss: 0.7948 - accuracy: 0.6000
Restored model, accuracy: 60.00%


In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#defined your model as `model`

# get class probabilities for test data
probabilities = model.predict(x_testcnn)

# get predicted classes by taking class with highest probability
predicted_classes = np.argmax(probabilities, axis=1)

# print classification report and confusion matrix
print(classification_report(y_test, predicted_classes))
print(confusion_matrix(y_test, predicted_classes))

1/1 [==============================] - 0s 26ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75         5
         1.0       0.33      0.50      0.40         2
         3.0       1.00      0.50      0.67         4
         4.0       0.57      1.00      0.73         4

    accuracy                           0.67        15
   macro avg       0.73      0.65      0.64        15
weighted avg       0.80      0.67      0.68        15

[[3 0 0 2]
 [0 1 0 1]
 [0 2 2 0]
 [0 0 0 4]]


In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

sm = Sequential()

sm.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
sm.add(Activation('relu'))
sm.add(Dropout(0.1))
sm.add(MaxPooling1D(pool_size=(8)))

sm.add(Conv1D(128, 5,padding='same',))                  #2
sm.add(Activation('relu'))
sm.add(MaxPooling1D(pool_size=(8)))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #3
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #4
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Flatten())
sm.add(Dense(8))                                        #5
sm.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(learning_rate=0.00005,epsilon=None,rho=0.9,decay=0.0)

In [ ]:
sm.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_7 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_5 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 activation_8 (Activation)   (None, 22, 128)           0         
                                                      

In [ ]:
sm.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
smhistory=sm.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Epoch 1/500
3/3 [==============================] - 1s 147ms/step - loss: 5.6303 - accuracy: 0.0714 - val_loss: 1.9080 - val_accuracy: 0.3333
Epoch 2/500
3/3 [==============================] - 0s 45ms/step - loss: 3.2891 - accuracy: 0.1786 - val_loss: 1.6641 - val_accuracy: 0.1333
Epoch 3/500
3/3 [==============================] - 0s 47ms/step - loss: 3.4457 - accuracy: 0.1071 - val_loss: 1.7854 - val_accuracy: 0.1333
Epoch 4/500
3/3 [==============================] - 0s 40ms/step - loss: 2.8020 - accuracy: 0.2321 - val_loss: 1.5780 - val_accuracy: 0.3333
Epoch 5/500
3/3 [==============================] - 0s 46ms/step - loss: 2.9157 - accuracy: 0.2143 - val_loss: 1.3981 - val_accuracy: 0.1333
Epoch 6/500
3/3 [==============================] - 0s 45ms/step - loss: 2.7025 - accuracy: 0.3571 - val_loss: 1.4357 - val_accuracy: 0.2667
Epoch 7/500
3/3 [==============================] - 0s 47ms/step - loss: 2.4252 - accuracy: 0.3393 - val_loss: 1.5278 - val_accuracy: 0.3333
Epoch 8/500
3/3 [==

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
p = np.argmax(sm.predict(x_testcnn), axis=-1)
print(classification_report(y_test,p))
print(confusion_matrix(y_test,p))


1/1 [==============================] - 0s 132ms/step
              precision    recall  f1-score   support

         0.0       0.50      0.20      0.29         5
         1.0       0.33      1.00      0.50         2
         3.0       0.00      0.00      0.00         4
         4.0       0.57      1.00      0.73         4

    accuracy                           0.47        15
   macro avg       0.35      0.55      0.38        15
weighted avg       0.36      0.47      0.36        15

[[1 1 0 3]
 [0 2 0 0]
 [1 3 0 0]
 [0 0 0 4]]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
loss, acc = sm.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1/1 [==============================] - 0s 32ms/step - loss: 0.8548 - accuracy: 0.4667
Restored model, accuracy: 46.67%
